In [1]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings(action='ignore')
import googlemaps
from datetime import datetime
import re # 정규식

In [89]:
mart = pd.read_csv('./data/서울특별시 대규모점포 인허가 정보.csv', encoding='utf-8-sig')
mart

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
0,3000000,2.019300e+18,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 506호 (낙원동),3140.0,낙원상가,2.019090e+13,U,40:00.0,그 밖의 대규모점포,198838.9130,452214.2110,대규모점포
1,3000000,2.019300e+18,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 (낙원동),3140.0,낙원상가,2.019090e+13,I,22:37.0,그 밖의 대규모점포,198838.9130,452214.2110,대규모점포
2,3000000,2.003300e+18,20030304,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,광장주식회사,2.007080e+13,I,59:59.0,시장,199837.4143,451974.1285,NaN
3,3030000,2.021300e+18,20130219,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 성동구 왕십리로 109 성수만세주유소 (성수동1가),4768.0,홈플러스익스프레스 뚝섬2점,2.021060e+13,U,06:00.0,구분없음,203841.2255,449475.5401,NaN
4,3000000,2.004300e+18,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,영풍문고,2.007070e+13,I,59:59.0,대형마트,198351.8187,451921.6030,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,3140000,2.011310e+18,20110630,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 양천구 중앙로 255-1 (신정동),NaN,홈플러스(주)익스프레스 신정점,2.021060e+13,U,02:00.0,구분없음,186934.1477,446336.6805,NaN
937,3040000,2.021300e+18,20210608,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 광진구 광나루로39길 11 (구의동),4977.0,구의 자이르네 판매시설,2.021090e+13,U,05:00.0,그 밖의 대규모점포,207754.5752,449270.2315,NaN
938,3060000,2.011310e+18,20110809,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중랑구 면목로 357 (면목동),131831.0,홈플러스(주)익스프레스 면목점,2.021060e+13,U,03:00.0,그 밖의 대규모점포,207708.3648,453525.7412,NaN
939,3060000,2.006310e+18,20061026,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중랑구 사가정로 332 (면목동),131822.0,홈플러스(주)면목점,2.021060e+13,U,03:00.0,복합쇼핑몰,207089.2166,453074.4768,NaN


In [90]:
mart = mart[mart['업태구분명'] == '대형마트']
mart = mart[mart['영업상태명']=='영업/정상']

In [91]:
mart

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
4,3000000,2.004300e+18,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,영풍문고,2.007070e+13,I,59:59.0,대형마트,198351.8187,451921.6030,NaN
7,3000000,2.005300e+18,20050127,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,종로세운상가,2.007070e+13,I,59:59.0,대형마트,NaN,NaN,NaN
75,3010000,2.004300e+18,20040601,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중구 청파로 426 (봉래동2가),100162.0,롯데쇼핑(주)롯데마트서울역점,2.017040e+13,I,59:59.0,대형마트,197243.2153,450655.1042,대규모점포
80,3010000,2.008300e+18,20080626,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중구 청계천로 400-0 (황학동),100899.0,주식회사 이마트 청계천점,2.020090e+13,U,09:00.0,대형마트,201823.9090,452076.8187,NaN
112,3020000,2.012300e+18,20120313,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 용산구 청파로 122 (한강로2가),140012.0,(주)농협유통하나로클럽용산점,2.020040e+13,U,40:00.0,대형마트,196813.2585,447881.9708,대규모점포
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,3130000,2.003310e+18,20030521,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 마포구 월드컵로 240 (성산동월드컵경기장내),NaN,홈플러스(주)월드컵점,2.021080e+13,U,05:00.0,대형마트,191010.3810,451972.1540,NaN
928,3240000,2.011320e+18,20110927,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 강동구 고덕로24길 50 (암사동),134854.0,홈플러스(주) 익스프레스 암사점,2.021060e+13,U,09:00.0,대형마트,211868.2443,450020.3552,NaN
931,3230000,2.007320e+18,20070709,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 송파구 올림픽로35가길 16 (신천동),138240.0,홈플러스(주)잠실점,2.021060e+13,U,09:00.0,대형마트,209039.1480,446007.9668,NaN
932,3100000,1.999310e+18,20011207,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 노원구 동일로204가길 12 (중계동),139926.0,홈플러스(주) 중계점,2.021060e+13,U,09:00.0,대형마트,205985.9533,459730.4378,NaN


In [5]:

mart = mart.drop(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소','도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '좌표정보(X)', '좌표정보(Y)', '점포구분명'], axis = 1)

In [6]:
mart.isnull().sum()

도로명주소    8
업태구분명    0
dtype: int64

In [7]:
mart = mart.dropna()

In [8]:
mart = mart.reset_index(drop=True)

In [9]:
for i in range(len(mart)):
    mart['도로명주소'][i] = re.sub("\(.*\)|\s-\s.*", '', mart['도로명주소'][i])
    print(mart['도로명주소'][i])

서울특별시 중구 청파로 426 
서울특별시 중구 청계천로 400-0 
서울특별시 용산구 청파로 122 
서울특별시 성동구 왕십리광장로 17 
서울특별시 광진구 광나루로56길 85 
서울특별시 광진구 아차산로 272 
서울특별시 동대문구 왕산로 214 
서울특별시 마포구 양화로 45 
서울특별시 동대문구 한천로 42 
서울특별시 중랑구 상봉로 118 
서울특별시 중랑구 동일로 932 
서울특별시 중랑구 망우로 336 
서울특별시 성북구 도봉로 17 
서울특별시 망우로 353-0 
서울특별시 성북구 종암로 167 
서울특별시 성북구  도봉로  17 
서울특별시 성북구 도봉로 17 신세계 이마트 미아점 
서울특별시 성북구 종암로 167  동 
서울특별시 강북구 도봉로33길 18 
서울특별시 강북구 삼양로27길 35-21 
서울특별시 강북구 삼양로 247 
서울특별시 강북구 도봉로67길 18 
서울특별시 강북구 도봉로 157 112호 
서울특별시 도봉구 노해로65길 4 
서울특별시 도봉구 도봉로 678 
서울특별시 노원구 노원로 330 
서울특별시 서초구 잠원로 69 
서울특별시 은평구 수색로 217 
서울특별시 은평구 은평로 111 
서울특별시 마포구  백범로  212 
서울특별시 마포구 백범로 212 
서울특별시 양천구 오목로 299 
서울특별시 양천로 559 
서울특별시 구로구 새말로 97 
서울특별시 구로구 디지털로32길 43 
서울특별시 금천구 두산로 71 
서울특별시 금천구 시흥대로 391 
서울특별시 금천구 시흥대로 391 
서울특별시 영등포구 선유로 156 
서울특별시 영등포구 선유로 138 
서울특별시 성북구 화랑로 76 
서울특별시 영등포구 영중로 125 
서울특별시 영등포구 여의동로3길 10 
서울특별시 영등포구 여의동로3길 10 
서울특별시 강서구 양천로 431 
서울특별시 강서구  화곡로  398 
서울특별시 구로구 경인로 661 신도림1차푸르지오 
서울특별시 방배로33길 29
서울특별시 서초구 양재대로 159 
서울특별시 서초구 매헌로 16 

In [10]:
mart

,도로명주소,업태구분명
0,서울특별시 중구 청파로 426,대형마트
1,서울특별시 중구 청계천로 400-0,대형마트
2,서울특별시 용산구 청파로 122,대형마트
3,서울특별시 성동구 왕십리광장로 17,대형마트
4,서울특별시 광진구 광나루로56길 85,대형마트
...,...,...
66,서울특별시 마포구 월드컵로 240,대형마트
67,서울특별시 강동구 고덕로24길 50,대형마트
68,서울특별시 송파구 올림픽로35가길 16,대형마트
69,서울특별시 노원구 동일로204가길 12,대형마트


In [11]:
mart_name = []
for name in mart['도로명주소']:
    mart_name.append(name)
    

In [12]:
gmaps = googlemaps.Client('개인 api 사용')
mart_lat = []
mart_lon = []
for i in mart_name:
    
    location = gmaps.geocode(i, language='ko') 
    
    latitude  = location[0]["geometry"]["location"]["lat"] 
    longitude = location[0]["geometry"]["location"]["lng"] 

    mart_lat.append(latitude)
    mart_lon.append(longitude)
    print(i)

서울특별시 중구 청파로 426 
서울특별시 중구 청계천로 400-0 
서울특별시 용산구 청파로 122 
서울특별시 성동구 왕십리광장로 17 
서울특별시 광진구 광나루로56길 85 
서울특별시 광진구 아차산로 272 
서울특별시 동대문구 왕산로 214 
서울특별시 마포구 양화로 45 
서울특별시 동대문구 한천로 42 
서울특별시 중랑구 상봉로 118 
서울특별시 중랑구 동일로 932 
서울특별시 중랑구 망우로 336 
서울특별시 성북구 도봉로 17 
서울특별시 망우로 353-0 
서울특별시 성북구 종암로 167 
서울특별시 성북구  도봉로  17 
서울특별시 성북구 도봉로 17 신세계 이마트 미아점 
서울특별시 성북구 종암로 167  동 
서울특별시 강북구 도봉로33길 18 
서울특별시 강북구 삼양로27길 35-21 
서울특별시 강북구 삼양로 247 
서울특별시 강북구 도봉로67길 18 
서울특별시 강북구 도봉로 157 112호 
서울특별시 도봉구 노해로65길 4 
서울특별시 도봉구 도봉로 678 
서울특별시 노원구 노원로 330 
서울특별시 서초구 잠원로 69 
서울특별시 은평구 수색로 217 
서울특별시 은평구 은평로 111 
서울특별시 마포구  백범로  212 
서울특별시 마포구 백범로 212 
서울특별시 양천구 오목로 299 
서울특별시 양천로 559 
서울특별시 구로구 새말로 97 
서울특별시 구로구 디지털로32길 43 
서울특별시 금천구 두산로 71 
서울특별시 금천구 시흥대로 391 
서울특별시 금천구 시흥대로 391 
서울특별시 영등포구 선유로 156 
서울특별시 영등포구 선유로 138 
서울특별시 성북구 화랑로 76 
서울특별시 영등포구 영중로 125 
서울특별시 영등포구 여의동로3길 10 
서울특별시 영등포구 여의동로3길 10 
서울특별시 강서구 양천로 431 
서울특별시 강서구  화곡로  398 
서울특별시 구로구 경인로 661 신도림1차푸르지오 
서울특별시 방배로33길 29
서울특별시 서초구 양재대로 159 
서울특별시 서초구 매헌로 16 

In [13]:
mart_lat

[37.5574069,
 37.5708733,
 37.5331353,
 37.5621482,
 37.5349372,
 37.5376127,
 37.5792928,
 37.5489656,
 37.579639,
 37.596438,
 37.613797,
 37.5967521,
 37.6108648,
 37.598081,
 37.602119,
 37.6108648,
 37.610891,
 37.602119,
 37.62153430000001,
 37.6179214,
 37.6254731,
 37.630752,
 37.62252,
 37.6516821,
 37.6648183,
 37.6466623,
 37.5102313,
 37.5795991,
 37.60077709999999,
 37.5424268,
 37.5424268,
 37.5257763,
 37.5578966,
 37.5070431,
 37.4843537,
 37.4704705,
 37.4686493,
 37.4686493,
 37.5276365,
 37.5260404,
 37.6017142,
 37.5277844,
 37.5181371,
 37.5181371,
 37.5642454,
 37.5584359,
 37.5094291,
 37.4912177,
 37.4619661,
 37.4665323,
 37.4934746,
 37.4992239,
 37.5112348,
 37.4917416,
 37.5131273,
 37.5386477,
 37.5546516,
 37.5745108,
 37.4519767,
 37.4686493,
 37.5456582,
 37.5704347,
 37.6241815,
 37.471482,
 37.5302698,
 37.4862512,
 37.5677584,
 37.5523222,
 37.5162099,
 37.639815,
 37.6157539]

In [14]:
mart_lon

[126.9694999,
 127.0213284,
 126.9647209,
 127.0379421,
 127.0957088,
 127.0726183,
 127.0479007,
 126.9134767,
 127.0691065,
 127.0935903,
 127.0775029,
 127.0902753,
 127.0296051,
 127.0905951,
 127.0332133,
 127.0296051,
 127.029824,
 127.0332133,
 127.0264792,
 127.0188996,
 127.0177185,
 127.0229047,
 127.0270305,
 127.046933,
 127.0435994,
 127.0709672,
 127.0069616,
 126.8987214,
 126.9204992,
 126.9533694,
 126.9533694,
 126.8703272,
 126.8622978,
 126.8902185,
 126.89788,
 126.8956692,
 126.8968106,
 126.8968106,
 126.8921603,
 126.8917337,
 127.0404475,
 126.9046359,
 126.9261297,
 126.9261297,
 126.8499416,
 126.8547981,
 126.8876466,
 126.9892572,
 127.03607,
 127.0355485,
 126.9891307,
 127.048424,
 127.0980274,
 127.1178672,
 127.1019609,
 127.1253669,
 127.1557517,
 127.0387578,
 126.900714,
 126.8968106,
 127.1422439,
 126.9747439,
 127.0273476,
 126.9826404,
 126.9017353,
 126.9412192,
 126.8981147,
 127.1350647,
 127.1030143,
 127.0686402,
 127.0937534]

In [15]:
mart['위도'] = mart_lat
mart['경도'] = mart_lon
mart

,도로명주소,업태구분명,위도,경도
0,서울특별시 중구 청파로 426,대형마트,37.557407,126.969500
1,서울특별시 중구 청계천로 400-0,대형마트,37.570873,127.021328
2,서울특별시 용산구 청파로 122,대형마트,37.533135,126.964721
3,서울특별시 성동구 왕십리광장로 17,대형마트,37.562148,127.037942
4,서울특별시 광진구 광나루로56길 85,대형마트,37.534937,127.095709
...,...,...,...,...
66,서울특별시 마포구 월드컵로 240,대형마트,37.567758,126.898115
67,서울특별시 강동구 고덕로24길 50,대형마트,37.552322,127.135065
68,서울특별시 송파구 올림픽로35가길 16,대형마트,37.516210,127.103014
69,서울특별시 노원구 동일로204가길 12,대형마트,37.639815,127.068640


In [16]:
mart['구'] = mart['도로명주소'].str.split(" ").str[1]

In [17]:
mart

,도로명주소,업태구분명,위도,경도,구
0,서울특별시 중구 청파로 426,대형마트,37.557407,126.969500,중구
1,서울특별시 중구 청계천로 400-0,대형마트,37.570873,127.021328,중구
2,서울특별시 용산구 청파로 122,대형마트,37.533135,126.964721,용산구
3,서울특별시 성동구 왕십리광장로 17,대형마트,37.562148,127.037942,성동구
4,서울특별시 광진구 광나루로56길 85,대형마트,37.534937,127.095709,광진구
...,...,...,...,...,...
66,서울특별시 마포구 월드컵로 240,대형마트,37.567758,126.898115,마포구
67,서울특별시 강동구 고덕로24길 50,대형마트,37.552322,127.135065,강동구
68,서울특별시 송파구 올림픽로35가길 16,대형마트,37.516210,127.103014,송파구
69,서울특별시 노원구 동일로204가길 12,대형마트,37.639815,127.068640,노원구


In [68]:
apart = pd.read_csv('./data/아파트(매매)__실거래가_20211113205311.csv')
apart

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,202104,15,"174,000",9,1988,언주로 103,NaN,-,-
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,54.98,202012,4,"146,000",1,1987,언주로 3,NaN,-,-
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202012,28,"155,400",3,1987,언주로 3,NaN,-,-
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202105,15,"195,000",5,1987,언주로 3,NaN,-,-
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,202108,11,"200,000",3,1987,언주로 3,NaN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48271,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),57.66,202102,27,"30,500",5,2003,동일로144길 74,NaN,-,-
48272,서울특별시 중랑구 중화동,274-75,274.0,75.0,한영(102),72.22,202105,4,"49,500",4,2003,동일로144길 74,NaN,-,-
48273,서울특별시 중랑구 중화동,274-76,274.0,76.0,한영(103),58.46,202107,1,"30,000",2,2003,동일로144길 74,NaN,-,-
48274,서울특별시 중랑구 중화동,274-76,274.0,76.0,한영(103),104.93,202108,20,"61,500",7,2003,동일로144길 74,NaN,-,-


In [69]:
apart['주소'] = apart['시군구']+apart['도로명']
apart = apart.drop(['시군구','번지','본번','부번','단지명','계약일','도로명','해제사유발생일','거래유형','중개사소재지'], axis = 1)
apart['구'] = apart['주소'].str.split(" ").str[1]


In [70]:
apart.isnull().sum()

전용면적(㎡)     0
계약년월        0
거래금액(만원)    0
층           0
건축년도        0
주소          0
구           0
dtype: int64

In [71]:
apart_location = []
for loc in apart['주소']:
    apart_location.append(loc)

In [75]:
len(apart_location)

48276

In [27]:
len(apart_location)

48276

In [76]:
gmaps = googlemaps.Client('개인 api 사용')


In [64]:

apart_lat = []
apart_lon = []
for i in apart_location:
    try:
        location = gmaps.geocode(i, language='ko') 
    
        latitude  = location[0]["geometry"]["location"]["lat"] 
        longitude = location[0]["geometry"]["location"]["lng"] 
        apart_lat.append(latitude)
        apart_lon.append(longitude)
    except IndexError:
        apart_lat.append('no')
        apart_lon.append('no')

In [77]:
lat1 = apart_lat[:]
lon1 = apart_lon[:]

In [78]:
while 'no' in lat1:
    lat1.remove('no')
len(lat1)
        

48058

In [84]:
while 'no' in lon1:
    lon1.remove('no')
len(lon1)
        

48058

In [80]:
apart_lat.count('no')

436

In [81]:
apart_lon.count('no')

0

In [83]:
len(apart_lon)

48058

In [85]:
len(apart_lat)

48494

In [ ]:
df = pd.read_csv('./data/Apart Deal.csv',encoding='utf-8-sig')
df

In [45]:
brower = webdriver.Chrome('./chromedriver.exe')

In [46]:
def SubwayInfoCrawling():
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    driver = webdriver.Chrome("chromedriver.exe", options=options)
    driver.get('http://www.seoulmetro.co.kr/kr/cyberStation.do')

    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    mapinfo = soup.find('div', 'mapInfo')
    lines = mapinfo.find_all('li')

    output = pd.DataFrame()
    for i in range(len(lines)):
        st_line = lines[i].span.text
        st_list = lines[i].div.text.split(',')
        for j in range(len(st_list)):
            st_name = st_list[j].strip()
            unit = pd.DataFrame({'st_name':[st_name],
                                 'st_line':[st_line]})
            output = pd.concat([output,unit], axis=0)

    output = output.reset_index(drop=True)
    driver.close()
    return output

In [47]:
st_info = SubwayInfoCrawling()

In [48]:
st_info

,st_name,st_line
0,소요산,1호선
1,동두천,1호선
2,보산,1호선
3,동두천중앙,1호선
4,지행,1호선
...,...,...
744,운양,김포도시철도
745,장기,김포도시철도
746,마산,김포도시철도
747,구래,김포도시철도


In [49]:
st_info['st_name'] = st_info['st_name'].apply(lambda x: x if x[-1]=='역' else x + '역')

In [50]:
sub_name = []
for name in st_info['st_name']:
    sub_name.append(name)
    

In [51]:
sub_name

['소요산역',
 '동두천역',
 '보산역',
 '동두천중앙역',
 '지행역',
 '덕정역',
 '덕계역',
 '양주역',
 '녹양역',
 '가능역',
 '의정부역',
 '회룡역',
 '망월사역',
 '도봉산역',
 '도봉역',
 '방학역',
 '창동역',
 '녹천역',
 '월계역',
 '광운대역',
 '석계역',
 '신이문역',
 '외대앞역',
 '회기역',
 '청량리역',
 '제기동역',
 '신설동역',
 '동묘앞역',
 '동대문역',
 '종로5가역',
 '종로3가역',
 '종각역',
 '시청역',
 '서울역',
 '남영역',
 '용산역',
 '노량진역',
 '대방역',
 '신길역',
 '영등포역',
 '신도림역',
 '구로역',
 '구일역',
 '개봉역',
 '오류동역',
 '온수역',
 '역곡역',
 '소사역',
 '부천역',
 '중동역',
 '송내역',
 '부개역',
 '부평역',
 '백운역',
 '동암역',
 '간석역',
 '주안역',
 '도화역',
 '제물포역',
 '도원역',
 '동인천역',
 '인천역',
 '가산디지털단지역',
 '독산역',
 '금천구청역',
 '광명역',
 '금천구청역',
 '석수역',
 '관악역',
 '안양역',
 '명학역',
 '금정역',
 '군포역',
 '당정역',
 '의왕역',
 '성균관대역',
 '화서역',
 '수원역',
 '세류역',
 '병점역',
 '세마역',
 '오산대역',
 '오산역',
 '진위역',
 '송탄역',
 '서정리역',
 '평택지제역',
 '평택역',
 '성환역',
 '직산역',
 '두정역',
 '천안역',
 '봉명역',
 '쌍용역',
 '아산역',
 '배방역',
 '온양온천역',
 '신창역',
 '서동탄역',
 '까치산역',
 '신정네거리역',
 '양천구청역',
 '도림천역',
 '신도림역',
 '문래역',
 '영등포구청역',
 '당산역',
 '합정역',
 '홍대입구역',
 '신촌역',
 '이대역',
 '아현역',
 '충정로역',
 '시청역',
 '을지로입구역',
 '을지로3가역',
 '을지로4가역

In [52]:
gmaps = googlemaps.Client('개인 api key 사용')
lat = []
lon = []
for i in sub_name:
    
    location = gmaps.geocode(i, language='ko') 
    
    latitude  = location[0]["geometry"]["location"]["lat"] 
    longitude = location[0]["geometry"]["location"]["lng"] 

    lat.append(latitude)
    lon.append(longitude)
    print(i)

소요산역
동두천역
보산역
동두천중앙역
지행역
덕정역
덕계역
양주역
녹양역
가능역
의정부역
회룡역
망월사역
도봉산역
도봉역
방학역
창동역
녹천역
월계역
광운대역
석계역
신이문역
외대앞역
회기역
청량리역
제기동역
신설동역
동묘앞역
동대문역
종로5가역
종로3가역
종각역
시청역
서울역
남영역
용산역
노량진역
대방역
신길역
영등포역
신도림역
구로역
구일역
개봉역
오류동역
온수역
역곡역
소사역
부천역
중동역
송내역
부개역
부평역
백운역
동암역
간석역
주안역
도화역
제물포역
도원역
동인천역
인천역
가산디지털단지역
독산역
금천구청역
광명역
금천구청역
석수역
관악역
안양역
명학역
금정역
군포역
당정역
의왕역
성균관대역
화서역
수원역
세류역
병점역
세마역
오산대역
오산역
진위역
송탄역
서정리역
평택지제역
평택역
성환역
직산역
두정역
천안역
봉명역
쌍용역
아산역
배방역
온양온천역
신창역
서동탄역
까치산역
신정네거리역
양천구청역
도림천역
신도림역
문래역
영등포구청역
당산역
합정역
홍대입구역
신촌역
이대역
아현역
충정로역
시청역
을지로입구역
을지로3가역
을지로4가역
동대문역사문화공원역
신당역
상왕십리역
왕십리역
한양대역
뚝섬역
성수역
건대입구역
구의역
강변역
잠실나루역
잠실역
잠실새내역
종합운동장역
삼성역
선릉역
역삼역
강남역
교대역
서초역
방배역
사당역
낙성대역
서울대입구역
봉천역
신림역
신대방역
구로디지털단지역
대림역
용답역
신답역
용두역
신설동역
대화역
주엽역
정발산역
마두역
백석역
대곡역
화정역
원당역
원흥역
삼송역
지축역
구파발역
연신내역
불광역
녹번역
홍제역
무악재역
독립문역
경복궁역
안국역
종로3가역
을지로3가역
충무로역
동대입구역
약수역
금호역
옥수역
압구정역
신사역
잠원역
고속터미널역
교대역
남부터미널역
양재역
매봉역
도곡역
대치역
학여울역
대청역
일원역
수서역
가락시장역
경찰병원역
오금역
당고개역
상계역
노원역
창동역
쌍문역
수유역
미아역
미아사거리역
길음역
성신여대입구역
한성대입구역
혜화역
동대문역
동대문역사문화공원역
충무로역
명동역
회현역
서울역
숙대입구역
삼각

In [54]:
lat

[37.947099,
 37.926664,
 37.914277,
 37.901673,
 37.892369,
 37.843216,
 37.81876,
 37.773956,
 37.759416,
 37.748362,
 37.7387295,
 37.724522,
 37.710507,
 37.689596,
 37.679441,
 37.66747,
 37.65276,
 37.644599,
 37.633114,
 37.6237325,
 37.6150815,
 37.601829,
 37.595518,
 37.58975600000001,
 37.581381,
 37.5781758,
 37.575326,
 37.573246,
 37.571983,
 37.570988,
 37.570436,
 37.570176,
 37.5657037,
 37.555946,
 37.540597,
 37.5298837,
 37.51408199999999,
 37.5125541,
 37.517346,
 37.5154133,
 37.5088099,
 37.5031784,
 37.496261,
 37.494698,
 37.49440000000001,
 37.4921505,
 37.4851398,
 37.482755,
 37.484085,
 37.486664,
 37.487626,
 37.488479,
 37.49038609999999,
 37.483301,
 37.470869,
 37.464689,
 37.4650414,
 37.466128,
 37.466809,
 37.46871,
 37.475414,
 37.4762789,
 37.48144,
 37.4659942,
 37.455645,
 37.416522,
 37.455645,
 37.435161,
 37.41957800000001,
 37.4026846,
 37.3900837,
 37.372237,
 37.354006,
 37.34338899999999,
 37.320648,
 37.300344,
 37.28399,
 37.2664398,
 37.

In [55]:
lon

[127.060681,
 127.054992,
 127.057158,
 127.056409,
 127.055757,
 127.061511,
 127.056676,
 127.044782,
 127.042272,
 127.044251,
 127.0458908,
 127.0472035,
 127.04738,
 127.0461945,
 127.045521,
 127.044295,
 127.047945,
 127.051406,
 127.058835,
 127.061817,
 127.0657675,
 127.067347,
 127.059615,
 127.057977,
 127.048958,
 127.0345907,
 127.0248495,
 127.016589,
 127.011708,
 127.001921,
 126.992148,
 126.983197,
 126.9768616,
 126.972317,
 126.971171,
 126.9648019,
 126.941687,
 126.9263706,
 126.9164095,
 126.9071288,
 126.8912061,
 126.8820367,
 126.86959,
 126.858504,
 126.844551,
 126.8236345,
 126.8116223,
 126.795654,
 126.782803,
 126.764474,
 126.753045,
 126.740549,
 126.7236421,
 126.707179,
 126.702884,
 126.693408,
 126.6807024,
 126.6685204,
 126.657165,
 126.642611,
 126.632598,
 126.6168922,
 126.882652,
 126.8895083,
 126.894003,
 126.884789,
 126.894003,
 126.902233,
 126.908476,
 126.9220496,
 126.9305068,
 126.9434895,
 126.948539,
 126.948401,
 126.948099,
 126

In [56]:
st_info['위도'] = lat
st_info['경도'] = lon
st_info

,st_name,st_line,위도,경도
0,소요산역,1호선,37.947099,127.060681
1,동두천역,1호선,37.926664,127.054992
2,보산역,1호선,37.914277,127.057158
3,동두천중앙역,1호선,37.901673,127.056409
4,지행역,1호선,37.892369,127.055757
...,...,...,...,...
744,운양역,김포도시철도,37.653851,126.683843
745,장기역,김포도시철도,37.644911,126.669600
746,마산역,김포도시철도,35.236120,128.577057
747,구래역,김포도시철도,37.645397,126.628708


In [62]:
st_info.to_csv('./data/subway.csv', header = True, index = False)

In [63]:
mart.to_csv('./data/mart.csv', header = True, index = False)